# Test URL Download

In [1]:
from rook.wcrawl.event import events

import threading
import _thread
import time

In [2]:
%run ./jupyter-test-support.ipynb

In [3]:
%run ./create_container.ipynb

Created [container]
  * app_root
  * blob_root_directory
  * blob_store
  * environment_name
  * event_publisher
  * mongodb_client
  * mongodb_credentials
  * queue_receiver
  * queue_sender
  * rabbitmq_connection
  * rabbitmq_url
  * url_download_handler
  * url_info_store
  * url_store


In [4]:
%%cell_testsuite

@cell_test
def test_send_urls(self):
    publisher = container.event_publisher()
    publisher.send_url_download('https://www.nytimes.com/')
    publisher.send_url_download('https://www.yahoo.com/')

.
----------------------------------------------------------------------
Ran 1 test in 0.096s

OK


## Try receiving
The utilities below have a hack to interupt the queue receiver after a delay.

In [9]:
def send_interrupt_after(seconds):
    def execute():
        time.sleep(seconds)
        _thread.interrupt_main()
    t = threading.Thread(target=execute)
    t.start()
    
def run_and_interrupt_after(func, seconds):
    send_interrupt_after(seconds)
    try:
        func()
    except KeyboardInterrupt:
        pass
    
def download_requests_for(seconds):
    q_receiver = container.queue_receiver()
    requests = []
    def request_handler(request):
        requests.append(request)
        
    run_and_interrupt_after(lambda: q_receiver.receive(events.URL_DOWNLOAD, request_handler), seconds)
    return requests

In [10]:
%%cell_testsuite

@cell_test
def test_receive(self):
    # Empty the queue
    download_requests_for(1)

    # Add a record
    publisher = container.event_publisher()
    publisher.send_url_download('https://www.bing.com/')
    
    requests = download_requests_for(1)
    print(requests)
    self.assertEqual(1, len(requests))
    self.assertEqual('https://www.bing.com/', requests[0]['url'])

[{'url': 'https://www.bing.com/'}]
.
----------------------------------------------------------------------
Ran 1 test in 10.122s

OK
